In [1]:
from sklearn.datasets import load_iris
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as  pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import GaussianNB
import warnings
warnings.filterwarnings('ignore')

In [2]:
raw_data= pd.read_csv("../../data/01-modified-data/cleaned_currency.csv")

In [3]:
btc=raw_data[raw_data['exchange']=='BTC/USD']

In [4]:

def cal_momentum(df):
    n = len(df)
    arr = []
    for i in range(0,5):
        arr.append('NaN')
    for j in range(5,n):
        momentum = df.close[j] - df.close[j-5] #Equation for momentum
        arr.append(momentum)
    return arr

mom=cal_momentum(btc)
btc['Momentum'] =mom

data_pctchange=btc.close.pct_change()
btc['Return'] = data_pctchange

In [5]:
def ROI(df,n):
    m = len(df)
    arr = []
    for i in range(0,n):
        arr.append('N')
    for j in range(n,m):
        roi= (df.close[j] - df.close[j-n])/df.close[j-n] 
        arr.append(roi)
    return arr

ROI30=ROI(btc,30)
btc['ROI-30D']=ROI30

In [6]:
#Function to Classify each day as a 1 or a 0
def class_y(df):
    n = len(df)
    arr = []
    for i in range(0,len(df)-1):
        if (100*((df.close[i+1]-df.open[i+1])/df.open[i+1]))>=.3:
            arr.append(1)
        else:
            arr.append(0)
    arr.append('0')
    return arr

class_btc=class_y(btc)

#Add Class to our dataframe
btc['class'] = class_btc

In [7]:
btc

,date,open,high,low,close,volume,exchange,M20,M5,Momentum,Return,ROI-30D,class
0,2021-09-01,47121.0,49131.0,46521.0,48846.0,4.844957e+08,BTC/USD,NaN,NaN,NaN,NaN,N,1
1,2021-09-02,48846.0,50407.0,48623.0,49273.0,6.025992e+08,BTC/USD,NaN,NaN,NaN,0.008742,N,1
2,2021-09-03,49273.0,51083.0,48340.0,50024.0,7.812154e+08,BTC/USD,NaN,NaN,NaN,0.015242,N,0
3,2021-09-04,50025.0,50564.0,49387.0,49936.0,3.602553e+08,BTC/USD,NaN,NaN,NaN,-0.001759,N,1
4,2021-09-05,49936.0,51928.0,49463.0,51807.0,4.353527e+08,BTC/USD,NaN,49977.2,NaN,0.037468,N,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
360,2022-08-27,20237.0,20378.0,19802.0,20036.0,3.499582e+08,BTC/USD,22607.10,20943.2,-1360.0,-0.009932,-0.159916,0
361,2022-08-28,20036.0,20155.0,19516.0,19555.0,3.124691e+08,BTC/USD,22394.05,20550.0,-1966.0,-0.024007,-0.177809,1
362,2022-08-29,19555.0,20417.0,19553.0,20291.0,4.638849e+08,BTC/USD,22250.75,20336.0,-1070.0,0.037637,-0.14203,0
363,2022-08-30,20288.0,20573.0,19542.0,19809.0,5.826412e+08,BTC/USD,22043.20,19985.6,-1752.0,-0.023754,-0.150193,1


In [8]:
X=btc.drop(btc.index[0:30])
X=X.drop(X.index[-1])
X=X.drop(['high','low','exchange','class'], axis=1)
X=X.set_index('date')

### Normalize data

In [9]:
def positivevalues(df):
    for column in df:
        if (df[column].min())<0:
            df[column]=(df[column]-df[column].min())

In [10]:
X_norm=(X-np.mean(X, axis=0))/np.std(X, axis=0)
df_norm=pd.DataFrame(X_norm)
df_norm.shape
df_norm.to_csv('x_naive_bayes_r.csv', index=False)

In [11]:
y=class_btc[30:-1]
y=pd.DataFrame(y)
y.shape
y.to_csv('y_naive_bayes_r.csv', index=False)

In [12]:
X=X_norm.to_numpy()
y=y.to_numpy()

In [13]:
x_train,x_test,y_train,y_test=train_test_split(X_norm,y,test_size=0.2)

In [14]:
from sklearn.naive_bayes import GaussianNB
GNB = GaussianNB()
GNB.fit(x_train,y_train)

GaussianNB()

In [15]:
y_pred=GNB.predict(x_test)

In [16]:
#model accuracy
from sklearn import metrics
print (metrics.accuracy_score(y_test, y_pred)) 

0.5671641791044776


In [17]:
def report(y,ypred):
    print("Accuracy:", accuracy_score(y,ypred)*100)
    print("Number of mislabeled points out of a total %d proints = %d" %(y.shape[0],(y!=ypred).sum()))

def print_model_summary():
    yp_train=GNB.predict(x_train)
    yp_test=GNB.predict(x_test)
    
    print("ACCURACY CALCULATION\n")

    print("TRAINING SET:")
    report(y_train,yp_train)

    print("TEST SET:")
    report(y_test,yp_test)

    print("\nCHECK FIRST 10 PREDICTIONS")
    print("TRAINING SET:")
    print(y_train[0:10])
    print(yp_train[0:10])
    print("ERRORS:",yp_train[0:10]-y_train[0:10])

    print("\nTEST SET:")
    print(y_test[0:10])
    print(yp_test[0:10])
    print("ERRORS:",yp_test[0:10]-y_test[0:10])


In [18]:
yp_train=GNB.predict(x_train)
yp_test=GNB.predict(x_test)
print_model_summary()

ACCURACY CALCULATION

TRAINING SET:
Accuracy: 58.42696629213483
Number of mislabeled points out of a total 267 proints = 33317
TEST SET:
Accuracy: 56.71641791044776
Number of mislabeled points out of a total 67 proints = 2157

CHECK FIRST 10 PREDICTIONS
TRAINING SET:
[[1]
 [1]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]]
[0 0 1 0 0 0 1 0 0 0]
ERRORS: [[-1 -1  0 -1 -1 -1  0 -1 -1 -1]
 [-1 -1  0 -1 -1 -1  0 -1 -1 -1]
 [ 0  0  1  0  0  0  1  0  0  0]
 [-1 -1  0 -1 -1 -1  0 -1 -1 -1]
 [ 0  0  1  0  0  0  1  0  0  0]
 [ 0  0  1  0  0  0  1  0  0  0]
 [ 0  0  1  0  0  0  1  0  0  0]
 [ 0  0  1  0  0  0  1  0  0  0]
 [ 0  0  1  0  0  0  1  0  0  0]
 [-1 -1  0 -1 -1 -1  0 -1 -1 -1]]

TEST SET:
[[0]
 [1]
 [0]
 [1]
 [0]
 [0]
 [0]
 [1]
 [1]
 [1]]
[0 1 0 0 0 0 0 0 0 0]
ERRORS: [[ 0  1  0  0  0  0  0  0  0  0]
 [-1  0 -1 -1 -1 -1 -1 -1 -1 -1]
 [ 0  1  0  0  0  0  0  0  0  0]
 [-1  0 -1 -1 -1 -1 -1 -1 -1 -1]
 [ 0  1  0  0  0  0  0  0  0  0]
 [ 0  1  0  0  0  0  0  0  0  0]
 [ 0  1  0  0  0  0  0  0  0  